In [16]:
import pandas as pd
import numpy as np
import os
import sys
sys.path.append('../')

# Load Student Data

In [9]:
from src.d01_data.student_data_api import StudentDataApi

sda = StudentDataApi()
student_df = sda.get_data(periods_list=["1819"]).set_index('studentno')
mask = student_df['grade'] == 'KG'
student_df = student_df.loc[mask]

student_df.head()

,r1_ranked_idschool,r1_listed_ranks,r1_programs,grade,r1_randomnumber,randomnumber,requestprogramdesignation,latitude,longitude,r2_ranked_idschool,...,Academic Score,Nhood SES Score,FRL Score,sibling,currentlpsibling,currentlp,msf,aaprek,aa,year
studentno,,,,,,,,,,,,,,,,,,,,,
10210002,[420],[1],['GE'],KG,[0.446027141579977],0.348868,0.0,37.747093,-122.429507,NaN,...,0.244898,0.342849,0.031694,[420],NaN,NaN,NaN,[420],[420],2019
10210005,"[435, 670]","[1, 2]","['GE', 'GE']",KG,"[0.9110051502117931, 0.111640986769763]",0.681393,0.0,37.718266,-122.480908,NaN,...,0.365385,0.766113,0.584975,NaN,NaN,NaN,NaN,NaN,[670],2019
10210006,"[729, 537, 680, 876, 729]","[1, 2, 3, 4, 5]","['SE', 'SE', 'SE', 'CE', 'GE']",KG,"[0.41175899240252206, 0.33197490658210305, 0.0...",0.694158,0.0,37.714647,-122.403722,NaN,...,0.380597,0.640319,0.660631,[729],NaN,NaN,NaN,NaN,NaN,2019
10210007,"[786, 569, 735, 435, 549, 413, 589, 479, 718, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","['JE', 'GE', 'GE', 'GE', 'GE', 'GE', 'GE', 'GE...",KG,"[0.9240686684326509, 0.591082980668961, 0.3187...",0.289696,0.0,37.780102,-122.442890,NaN,...,0.022388,0.504279,0.263899,NaN,NaN,NaN,NaN,NaN,[735],2019
10210010,"[796, 420, 505, 722, 589, 618, 876, 718, 493, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","['GE', 'SE', 'GE', 'GE', 'GE', 'SE', 'CE', 'GE...",KG,"[0.0774816984940387, 0.826669827283934, 0.8127...",0.200155,0.0,37.758470,-122.442804,NaN,...,0.000000,0.288537,0.000000,NaN,NaN,NaN,NaN,NaN,[505],2019


# Load Assignment Results

In [11]:
equity_tiebreaker = 'ctip1'
diversity_category_col = 'Diversity_Category3'
program_cutoff = 'program_cutoff'

def get_specific_program_cutoff(x, diversity_category):
    x_list = x[1:-1].split()
    return float(x_list[diversity_category])

def check_tiebreaker(row):
    diversity_category = row[diversity_category_col]
    cut_off = get_specific_program_cutoff(row[program_cutoff], diversity_category)
    if cut_off > 1 + 2 + 4:
        return "sibiling"
    elif cut_off > 1 + 2 :
        return "equity"
    elif cut_off > 1:
        return "zone"
    else:
        return "lottery"


In [13]:
assignments_dir = "/share/data/school_choice_equity/local_runs/Assignments/"
trial = ""

for filename in sorted(os.listdir(assignments_dir+trial)):
    try:
        print(filename)
        assignment_df = pd.read_csv(assignments_dir+trial+filename).set_index('studentno')
        break
    except:
        pass    

    
assignment_df['tiebreaker'] = assignment_df.apply(lambda row: check_tiebreaker(row), axis=1, raw=False)
assignment_df[equity_tiebreaker] = student_df[equity_tiebreaker].reindex(assignment_df.index)
assignment_df.head()

Assignment_CTIP1_round_merged123_policyMedium1-GuardRails0-prefLength07_tiesSTB_prefExtend0_iteration0.csv


,Unnamed: 0,Diversity_Category3,programno,programcodes,rank,inZone,designation,program_cutoff,assigned_utility,In-Zone Rank,tiebreaker,ctip1
studentno,,,,,,,,,,,,
10210005,0,1,25.0,478-JE-KG,2.0,NaN,0.0,[1.8187873 1.15003326 3.29639322],3.896475,2,zone,0.0
10210006,1,1,94.0,729-SE-KG,1.0,NaN,0.0,[1.17455435 1.77810531 3.43627379],7.004528,1,zone,0.0
10210007,2,0,76.0,479-GE-KG,1.0,NaN,0.0,[1.73677861 1.04777155 3.04105669],5.072171,1,zone,0.0
10210010,3,0,88.0,505-GE-KG,5.0,NaN,0.0,[1.18241297 0. 0. ],3.614739,2,zone,0.0
10210039,4,2,125.0,820-GE-KG,8.0,NaN,1.0,[1.00475605 0. 0. ],0.964402,6,lottery,0.0


In [19]:
evaluation_columns = [diversity_category_col, 'rank', 'designation', 'In-Zone Rank', 'tiebreaker', equity_tiebreaker]
def q1(x):
    return np.quantile(x, .25)
def q2(x):
    return np.quantile(x, .5)
def q3(x):
    return np.quantile(x, .75)
    

assignment_df[evaluation_columns].groupby([diversity_category_col, 'tiebreaker']).agg({'rank':['count','mean', 'min', q1, q2, q3, 'max'],
                                                                                            equity_tiebreaker: 'mean'})

rank                                     \
                               count      mean  min   q1   q2   q3  max   
Diversity_Category3 tiebreaker                                            
0                   lottery      341  5.472141  1.0  3.0  6.0  8.0  8.0   
                    sibiling       2  2.000000  2.0  2.0  2.0  2.0  2.0   
                    zone        1252  5.103035  1.0  2.0  5.0  8.0  8.0   
1                   lottery      443  4.422122  1.0  2.0  4.0  8.0  8.0   
                    zone        1152  4.368924  1.0  2.0  4.0  8.0  8.0   
2                   equity       216  3.226852  1.0  1.0  2.0  5.0  8.0   
                    lottery      441  5.142857  1.0  2.0  5.0  8.0  8.0   
                    zone         925  4.845405  1.0  2.0  5.0  8.0  8.0   

                                   ctip1  
                                    mean  
Diversity_Category3 tiebreaker            
0                   lottery     0.000000  
                    sibiling    0.000000  
                    zone        0.005632  
1                   lottery     0.000000  
                    zone        0.017513  
2                   equity      0.990741  
                    lottery     0.260766  
                    zone        0.410372